## Installing the transformers including gemma3

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-e9_iix3a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-e9_iix3a
  Running command git checkout -q 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Resolved https://github.com/huggingface/transformers to commit 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936429 sha256=75ca1177beda73d553e501f9de76fcaed12e6b88089d5ecc9512c491a64e8942
  Stored in directory: /tmp/pip-ephem-wheel-cache-ol6ej_ez/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transf

## Importing the libraries

In [3]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM

## Step 1: Load the Gemma 3 Model

In [4]:
ckpt = "google/gemma-3-1b-pt"
# google/gemma-3-1b-it
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

## Step 2: Define Function to Generate Code Explanation

In [5]:
def explain_code(code_snippet):
    """
    This function takes a Python code snippet as input
    and generates an explanation using the Gemma 3 model.
    """
    prompt = f"""Explain the following Python code in simple terms:

```python
{code_snippet}
```

Explanation:"""

    # Tokenize input and move to model's device
    model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Get input length to extract only the new generated tokens
    input_len = model_inputs["input_ids"].shape[-1]

    # Step 3: Generate Explanation using the Model
    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=150, do_sample=False)
        generation = generation[0][input_len:]

    # Decode the generated output
    decoded = tokenizer.decode(generation, skip_special_tokens=True)
    return decoded

## Step 4: Provide an Example Code Snippet


In [6]:
code = """


import numpy as np
arr = np.array([1, 2, 3, 4])
print(arr.mean())


"""

## Step 5: Get and Print the Explanation

In [7]:
explanation = explain_code(code)

## Step 6: Display the Result in Markdown Format

In [8]:
print("""
# Code Explanation

""" + explanation + """

""")


# Code Explanation



The code above calculates the mean of the array. The mean is the average of the elements in the array. The mean is calculated by adding up all the elements in the array and dividing by the number of elements in the array.

The code above calculates the mean of the array. The mean is the average of the elements in the array. The mean is calculated by adding up all the elements in the array and dividing by the number of elements in the array.

The code above calculates the mean of the array. The mean is the average of the elements in the array. The mean is calculated by adding up all the elements in the array and dividing by the number of elements in the array.

The code above calculates the mean of the


